# Importing Spark

In [13]:
# import findspark
# findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Spark").getOrCreate()
sc = spark.sparkContext

# Loading the data

In [14]:
df_ratings = spark.read\
    .option("delimiter", "\t")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('data/u.data')

# Exercice 1 - constructing the recommender system

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [16]:
df_ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [17]:
# split
training, test = df_ratings.randomSplit([0.8, 0.2])
training.show()
test.show()

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|      2|     3|876893171|
|      1|      5|     3|889751712|
|      1|      6|     5|887431973|
|      1|      7|     4|875071561|
|      1|      9|     5|878543541|
|      1|     10|     3|875693118|
|      1|     11|     2|875072262|
|      1|     13|     5|875071805|
|      1|     14|     5|874965706|
|      1|     16|     5|878543541|
|      1|     17|     3|875073198|
|      1|     20|     4|887431883|
|      1|     21|     1|878542772|
|      1|     23|     4|875072895|
|      1|     24|     3|875071713|
|      1|     25|     4|875071805|
|      1|     26|     3|875072442|
|      1|     27|     2|876892946|
|      1|     28|     4|875072173|
|      1|     30|     3|878542515|
+-------+-------+------+---------+
only showing top 20 rows

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|      1|     5|87496

In [24]:
# fit ALS
als = ALS(maxIter=5,regParam=0.05, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop",rank=10)
# nombre de lignes ou colonnes
model = als.fit(training)


In [25]:
# evaluate
predictions = model.transform(test)
predictions.show(5)

+-------+-------+------+---------+----------+
|user_id|item_id|rating|timestamp|prediction|
+-------+-------+------+---------+----------+
|    330|    148|     4|876544781|  4.521793|
|    224|    148|     3|888104154| 3.1988578|
|      1|    148|     2|875240799| 2.6778774|
|     15|    148|     3|879456049| 2.4196055|
|    677|    148|     4|889399265|   4.22349|
+-------+-------+------+---------+----------+
only showing top 5 rows



In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
#RegressionEvaluator
evaluator = RegressionEvaluator().setLabelCol('rating').setPredictionCol('prediction').setMetricName('rmse')
#evaluator = RegressionEvaluator(
 #          metricName="areaUnderROC",
 #          labelCol="rating",
 #          predictionCol="prediction")


RMSE = evaluator.evaluate(predictions)
print(RMSE)

0.9587225479306362


In [ ]:
# Generate top 10 movie recommendations for each user
# Generate top 10 user recommendations for each movie
model.recommendForAllUsers(10)
model.recommendForAllItems(10)

In [43]:
## todo récupérer un  vecteur de film et un vecteur d'un utilisateur
## faire le produit scalaire entre les deux pouyr voir la prédiction
## puis faire la même chose  avec spark
user_id=50
item_id=100
model.userFactors.filter(model.userFactors['id']==user_id).collect.show(5)
model.itemFactors.filter(model.itemFactors['id']==item_id).show(5)
user_vector = model.userFactors.filter(model.userFactors['id']==item_id).flatten()
item_vector  = model.itemFactors.filter(model.itemFactors['id']==item_id).flatten()

import numpy as np
np.dot(user_npy, item_npy)


from  pyspark.sql.types import ArrayType,StructField, StructType, StringType, IntegerType, decimalType

data=[(10,10)]
schema =  StructType([
    
])
model.transform(df).show()

+---+--------------------+
| id|            features|
+---+--------------------+
| 50|[0.1671989, 0.132...|
+---+--------------------+

+---+--------------------+
| id|            features|
+---+--------------------+
|100|[-0.5522704, -0.1...|
+---+--------------------+



AttributeError: 'DataFrame' object has no attribute 'flatten'